# 第11章 ファンクターからアプリカティブファンクターへ

### 帰ってきたファンクター

ファンクターは文脈を持った値とみなすことができる  
Maybe値は「計算が失敗したかもしれない」という文脈を、リストは「複数の値を同時に取るかもしれない」という文脈を持つ  

fmapはこういった文脈を保ったまま関数を値に適用する  

### ファンクターとしてのI/Oアクション

IOもファンクターの一種  

```
instance Functor IO where
    fmap f action = do
        result <- action
        return (f result)
```



In [2]:
main = do
    line <- fmap reverse getLine
    putStrLn $ "You said " ++ line ++ " backwards!"
    putStrLn $ "Yes, you really said " ++ line ++ " backwards!"
    

In [3]:
main

You said egohegohegoh backwards!
Yes, you really said egohegohegoh backwards!

In [5]:
-- ファンクターの中身を１つではなく複数関数使いたいなら関数合成がよさげ

import Data.Char
import Data.List

main = do
    line <- fmap (intersperse '-' . reverse . map toUpper) getLine
    putStrLn line

main

### ファンクターとしての関数

`(->) r` もファンクター  
`r -> a` は `(->) r a` と書き換えられる  

実装  

```
instance Functor ((->) r) where
    fmap f g = (\x -> f (g x))
```

fmapの型  

```
fmap :: (a -> b) -> f a -> f b

fmap :: (a -> b) -> ((->) r a) -> ((->) r b)

fmap :: (a -> b) -> (r -> a) -> (r -> b)
```

aからbへの関数と、rからaへの関数を引数に取り、rからbへの関数を返す  
つまり関数合成  
そう考えるとインスタンス宣言はこう書ける  

```
instance Functor ((->) r) where
    fmap = (.)
```

In [7]:
:m + Control.Monad.Instances

In [8]:
:t fmap (*3) (+100)

fmap (*3) (+100) :: forall b. Num b => b -> b

In [11]:
fmap (* 3 ) (+ 100) 1

303

In [13]:
:t (.)

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

In [14]:
:t fmap

fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

fmapは関数を取って「元の関数に似てるけどファンクター値を取って、ファンクター値を返す関数」を返す関数  
関数 a -> bを受け取って、関数 f a -> f bを返す操作のことを 持ち上げ `lifting` という  


In [15]:
:t fmap (*2)

fmap (*2) :: forall (f :: * -> *) b. (Functor f, Num b) => f b -> f b

In [16]:
:t fmap (replicate 3)

fmap (replicate 3) :: forall (f :: * -> *) a. Functor f => f a -> f [a]

fmapについての２通りの考え方  

- fmapは関数とファンクター値を取って、その関数でファンクター値を写して返すもの
- fmapは値から値への関数を取って、それをファンクター値からファンクター値への関数に持ち上げたものを返す関数である


In [17]:
fmap (replicate 3) [1,2,3,4]

[[1,1,1],[2,2,2],[3,3,3],[4,4,4]]

In [18]:
fmap (replicate 3) (Just 4)

Just [4,4,4]

In [19]:
fmap (replicate 3) Nothing

Nothing

次回

## ファンクター則

デュエルスタンバイ

fmap fをファンクターに適用したらそれはファンクターの中身にfを適用するべきで、それ以上のことはしてはいけない  
この挙動はファンクター則に記述されている  

Functorのインスタンスは、ファンクター則の２性質を満たしている必要がある  

### 第一法則

idでファンクター値を写した場合、ファンクター値が変化してはいけない  

`fmap id = id`  



In [1]:
fmap id (Just 3)

Line 1: Functor law
Found:
fmap id
Why not:
id

Just 3

In [2]:
id Just 3

Line 1: Evaluate
Found:
id Just
Why not:
Just

Just 3

In [3]:
fmap id [1..5]

Line 1: Functor law
Found:
fmap id
Why not:
id

[1,2,3,4,5]

```
instance Functor Maybe where
    fmap f (Just x) = Just (f x)
    fmap f Nothing = Nothing
```

### 第二法則

関数合成と写す操作の間の関係  
「fとgの合成関数でファンクター値を写したもの」と「まずg, 次にfでファンクター値を写したもの」が等しいことを要求する  

```
fmap (f . g) = fmap f . fmap g
fmap (f . g) x = fmap f (fmap g x)

-- Justの場合

fmap (f . g) (Just x)
Just (f (g x))

fmap f (fmap g (Just x))
fmap f (Just (g x)
fmap (Just (f (g x))
```


In [4]:
data CMaybe a = CNothing | CJust Int a deriving (Show)

In [5]:
CJust 0 "haaha"

CJust 0 "haaha"

In [6]:
instance Functor CMaybe where
    fmap f CNothing = CNothing
    fmap f (CJust counter x) = CJust (counter + 1) (f x)

In [7]:
fmap (++"ha") (CJust 0 "ho")

CJust 1 "hoha"

In [9]:
fmap id (CJust 0 "haha")
-- 余計なこと (counterを1プラスすること)をやっているため第一法則を満たさない

Line 1: Functor law
Found:
fmap id
Why not:
id

CJust 1 "haha"

### まとめ

- ファンクターとは？
  - 文脈を持った値
  - 第一法則 (id (\x => x)で写した時値が変化しないこと)を満たす
  - 第二法則 (関数合成後の関数で写した時と、合成前の関数を順次写した時で値が変化しないこと)を満たす
  

## アプリカティブファンクタを使おう

アプリカティブファンクター: ファンクターの強化版（強い)

今までは1引数の関数を渡していたが、2引数の関数(例えば \*)とか使ったらどうなるのかという問から始まる

In [10]:
fmap (*) (Just 3)

No instance for (Show (a0 -> a0))
  (maybe you haven't applied enough arguments to a function?)
  arising from a use of `print'
In the first argument of `print', namely `it'
In a stmt of an interactive GHCi command: print it

In [11]:
:t fmap (++) (Just "hey")

fmap (++) (Just "hey") :: Maybe ([Char] -> [Char])

他引数関数でファンクター値を移すと、関数が入ったファンクター値が返る  

Q. 例えば、ファンクター値 `Just (3 *)` と　`Just 5` があったとして、前者から関数を取り、後者の値に適用させたくなったらどうする？  
A. ふつうのファンクターではこれはムリ  
ふつうのファンクターでできるのは、通常の関数でファンクターの中の値を写すことだけ  

### Applicativeちゃんと仲良くしてあげてね！

Control.Applicativeモジュール  

型クラスApplicativeは`pure`と `<*>` を定義している

```
class (Functor f) => Applicative f where
    pure :: a -> f a
    (<*>) :: f (a -> b) -> f a -> f b
```

MaybeのApplicativeインスタンス  

```
instance Applicative Maybe where
    pure = Just
    Nothing <*> _ = Nothing
    (Just f) <*> something = fmap f something

-- pure = Just は pure x = Just xと同じ
```

In [12]:
Just(+3) <*> Just (9)

Line 1: Redundant bracket
Found:
(9)
Why not:
9

Just 12

In [13]:
pure (+3) <*> Just 3

Just 6

In [14]:
Nothing <*> Just "woot"

Nothing

### まとめ

- アプリカティブファンクターとは
  - ファンクターの強化版
  - 値として関数を持つことが出来る
  - その関数を適用させることが出来る

というところ？

### アプリカティブスタイル

`pure f <*> x  <*> y <*> ...`のこと  

もっと理解して便利に使うポイント  

`pure f <*> x` は `fmap f x` と等しい  
アプリカティブ則の１つ(後半にまた出てくるっぽい)  

fmapと等価な演算子<$>  

```
(<$>) :: (Functor f) => (a -> b) -> f a -> f b
    f <$> x = fmap f x
```

例) 関数fを3つのアプリカティブ値の引数に適用したい時下記のように書ける

```
f <$> x <*> y <*> z
```



In [16]:
-- リストの例

[(*0), (+100), (^2)] <*> [1,2,3]

[0,0,0,101,102,103,1,4,9]

In [17]:
[ x * y | x <- [ 2,5,10], y <- [8,10, 11]]

[16,20,22,40,50,55,80,100,110]

In [19]:
(*) <$> [2,5,10] <*> [ 8,10,11]

-- 実は同じ！

[16,20,22,40,50,55,80,100,110]

次回

### IOもアプリカティブファンクターだよ！

デュエルスタンバイ！